# Optimization in JuMP

Code: https://github.com/JuliaOpt/JuMP.jl Docs: http://www.juliaopt.org/JuMP.jl

From the documentation:

"JuMP is a domain-specific modeling language for mathematical optimization embedded in Julia. It currently supports a number of open-source and commercial solvers (see below) for a variety of problem classes, including linear programming, mixed-integer programming, second-order conic programming, semidefinite programming, and nonlinear programming."
* Simple syntax
* Fast
* Solver independent language
* Supports many solvers
 - Including: Artelys Knitro, Bonmin, Cbc, Clp, Couenne, CPLEX, ECOS, FICO Xpress, GLPK, Gurobi, Ipopt, MOSEK, NLopt, and SCS.
* Written in Julia

Note: Not all solvers support all sorts of sets and constraints. This means that you should select a solver after your requirements. But also that you can use very efficient solvers for special problems, such as LP or QP. 

### In JuMP, each model is connected with an optimizer

In [ ]:
using JuMP, Ipopt
model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

Variables are defined using `@variable` and connected to a model.
Vectors can be defined using `x[1:n]`, and simple constraints can optionally be added

In [ ]:
n = 10
@variable(model, x[1:n] >= 0)

### Adding Constraints
Simple constraints can be added uisng the built in Julia operators, and optionally be given names using

`@constraint(model, [name,] constraint)`

or 

`@constraint(model, [name,] expression in MOI.SomeSetHere())`

In [ ]:
@constraint(model, firstbig, 1 <= x[10] <= 10)

And a set of constraints can be created in a simple way

In [ ]:
@constraint(model, first3[k=1:3], x[k] <= 10*k)

Constraints can then be inspected as expected

In [ ]:
firstbig

In [ ]:
first3

And the full model so far can be printed

In [ ]:
model

Vector constraints can be added with the broadcast syntax `.`

In [ ]:
using Random
Random.seed!(2)
m = 3

A = randn(m,n)
b = randn(m)

@constraint(model, A*x .== b);

Expressions can be built up using `@expression`

In [ ]:
C = randn(m,n)
d = randn(m)

ineq = @expression(model, C*x - d);

And add constraints for expresions in sets

In [ ]:
@constraint(model, ineq in MOI.Nonnegatives(m));

Or we could simply have written

`@constraint(model, C*x .<= d)`

### Objective
Simple cost functions can be defined by

`@objective(model, sense, expression)`

In [ ]:
M = randn(n,n)
Q = M*M'
p = randn(n)

@objective(model, Min,  x'*Q*x - p'*x);

### Solving
the solver is called using `optimize!`
and values are accessed using `value`

In [ ]:
optimize!(model)

@show termination_status(model)
sol = value.(x)

We can also ask the solver for a range of data, such as objective_value, dual variables and so on.

See http://www.juliaopt.org/JuMP.jl for details

In [ ]:
@show objective_value(model)

@show A*sol-b         # Should be rougly zero

# We can evaluate expressions
@show value.(ineq);    # Should be nonnegative

### General nonlinear programming is also supported by some solvers

These objectives and constraints can be entered as

`@NLobjective(model, Min, my_function(x, y))`

and

`@NLconstraint(model, x[1] * x[2] * x[3] * x[4] >= 25)`